In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('dataset.csv')
df.describe()

,Year,Quarter,Births
count,52.000000,52.000000,52.000000
mean,2006.000000,2.500000,14903.423077
std,3.778162,1.128942,943.667280
min,2000.000000,1.000000,13151.000000
25%,2003.000000,1.750000,14208.250000
50%,2006.000000,2.500000,14881.000000
75%,2009.000000,3.250000,15646.500000
max,2012.000000,4.000000,16968.000000


In [3]:
df.columns

Index(['Year', 'Quarter', 'Births'], dtype='object')

In [ ]:
features = df[['Time','Premise','Age','Gender']].values
label = df[['Crime Code']].values.reshape(-1,1)
label

In [ ]:
label.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2)

In [ ]:
y_test.shape

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
features[:, [0]].mean()

In [ ]:
x_train[:, [0]].mean()

In [ ]:
x_train[0]

In [ ]:
#Constant
#PlaceHolder
input_tensor = tf.placeholder(tf.float32, name="input")
label_tensor = tf.placeholder(tf.float32, name="output")

#Variable
parameters = {
    'W1' : tf.Variable(
        tf.random.normal((4,64)),
        dtype=tf.float32
    ),
    'B1' : tf.Variable(
        tf.random.normal((1,64)),
        dtype=tf.float32
    ),
    'W2' : tf.Variable(
        tf.random.normal((64,64)),
        dtype=tf.float32
    ),
    'B2' : tf.Variable(
        tf.random.normal((1,64)),
        dtype=tf.float32
    ),
    'W3' : tf.Variable(
        tf.random.normal((64,3)),
        dtype=tf.float32
    ),
    'B3' : tf.Variable(
        tf.random.normal((1,3)),
        dtype=tf.float32
    ),
}


In [ ]:
def forward(x):
    z1 = tf.matmul(x, parameters['W1']) + parameters['B1']
    a1 = tf.nn.sigmoid(z1)
    
    z2 = tf.matmul(a1, parameters['W2']) + parameters['B2']
    a2 = tf.nn.sigmoid(z2)
    
    z3 = tf.matmul(a2, parameters['W3']) + parameters['B3']
    a3 = tf.nn.sigmoid(z3)
    
    return a3

In [ ]:
def forward_pass(input_tensor,params,n_layer=3):
    a = input_tensor
    for i in range(n_layer):
        W = params[f'W{i+1}']
        B = params[f'B{i+1}']

        z = tf.matmul(a,W) + B #update weight
        if i == n_layer-1:
            a = tf.nn.sigmoid(z)
        else:
            a = tf.nn.sigmoid(z)

    return a

In [ ]:
with tf.Session() as sess:
    logits_tensor = forward_pass(input_tensor, parameters)
    loss_tensor = tf.losses.mean_squared_error(label_tensor, logits_tensor)

    true_preds_tensor = tf.equal(tf.argmax(logits_tensor, axis=1), tf.argmax(label_tensor, axis=1))
    acc_tensor = tf.reduce_mean(tf.cast(true_preds_tensor, tf.float32))

    optimizer = tf.train.GradientDescentOptimizer().minimize(loss_tensor)
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        sess.run(
            optimiser,
            feed_dict = {
                input_tensor: x_train,
                label_tensor: y_train
            }
        )

        #Validasi
        if i%10 == 0:
            loss,acc = evaluate_accuracy(y_test,sess.run(
            logits_tensor,
            feed_dict = {
                input_tensor: x_test,
                label_tensor: y_test
            }
        ))
        if i % 100 == 0:
            print(f'EPOCH: {i} | ACCURACY: {acc*100 : .2f}% |ERROR: {loss} ')
      
  #print(f'ACCURACY: {acc*100 : .2f}%')